In [1]:
# uncomment bellow to install dependences

# !pip install portalocker>=2.0.0
# !pip install scikit-plot
# !pip install scipy==1.11.4
# !pip install lime
# !pip install torchtext

In [2]:
import torchtext
import torch

In [3]:
torch.__version__

'2.2.1+cu118'

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("FolhaArticles.csv", sep='\t')

In [6]:
df.head()

,Title,Content,Url,Published,categories
0,Visões da batalha na Rússia e no Ocidente são ...,Quatro décadas de Guerra Fria e o renovado con...,https://www1.folha.uol.com.br/mundo/2019/06/vi...,2019-06-06 02:00:00.0000000,mundo
1,Bolsonaro é intimado pela PF sobre golpe e pod...,A Polícia Federal intimou Jair Bolsonaro (PL) ...,https://www1.folha.uol.com.br/poder/2024/02/pf...,2024-02-19 12:45:00.0000000,poder
2,Hayao Miyazaki passa a vida a limpo em O Menin...,Como vocês vivem? A pergunta parece ser feita ...,https://www1.folha.uol.com.br/ilustrada/2024/0...,2024-02-19 07:00:00.0000000,ilustrada
3,Série produzida pelo Estúdio Folha ganha Prêmi...,"A série ""Caminhos Proibidos"", produzida pelo E...",https://estudio.folha.uol.com.br/estudio/2024/...,2024-01-18 16:49:00.0000000,estudio
4,"Como é 'Crystal', do Cirque du Soleil, que vem...",Uma mulher expressa sua angústia dançando sobr...,https://www1.folha.uol.com.br/ilustrada/2024/0...,2024-02-19 15:15:00.0000000,ilustrada


In [7]:
df.dropna(inplace=True)

In [8]:
categories = [ 'celebridades', 'ciencia', 'cinema', 'comida', 'educacao', 'eleicoes', 'esporte', 'mercado', 'poder']
df = df[df['categories'].isin(categories)]

In [9]:
df['Category'] = [categories.index(x) for x in df.categories]

In [10]:
df[['Category', 'Content']].shape

(64400, 2)

In [11]:
df_train = df[['Category', 'Content']][:54000]
df_test = df[['Category', 'Content']][54000:]

In [12]:
df_train.shape, df_test.shape

((54000, 2), (10400, 2))

In [13]:
df_train = df_train.values.tolist()
df_test = df_test.values.tolist()

In [14]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

def build_vocab(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocab([df_train, df_test]), specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

In [15]:
len(vocab.get_itos())

352993

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vectorizer = CountVectorizer(vocabulary=vocab.get_itos(), tokenizer=tokenizer)
def vectorize_batch(batch):
    Y, X = list(zip(*batch)) # agrupa labels em uma tupla e textos em outra tupla (exemplo abaixo)
    X = vectorizer.transform(X).todense()
    return torch.tensor(X, dtype=torch.float32).cuda(), torch.tensor(Y).cuda()


In [17]:

from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

train_loader = DataLoader(df_train, batch_size=256, collate_fn=vectorize_batch)
test_loader  = DataLoader(df_test, batch_size=256, collate_fn=vectorize_batch)

In [18]:
df_test[0]

[7,
 'Além da experiência e da maturidade, uma das vantagens de se chegar aos 50 anos é que, normalmente, a pessoa conseguiu acumular algum patrimônio ao longo da vida. Esse é um fator importante para quem chegou a essa idade e nunca se planejou para a aposentadoria.  Começar a pensar na previdência nessa idade demanda alguns sacrifícios a mais e muita organização. A pessoa precisa rever todos os seus gastos para encontrar um espaço para poupar dinheiro.  Calcule o que falta para sua independência financeiraFerramenta exclusiva da Folha permite estimar poupança necessáriaÉ preciso ter em mente que não tem como recuperar o tempo que passou. Então, os aportes mensais para essa finalidade terão de ser maiores nessa fase da vida, se o indivíduo quiser manter o padrão de vida que tinha antes de parar de trabalhar.  Falando nisso, o analista de renda fixa André Alírio, da Nova Futura Investimentos, lembra que a realidade do Brasil não comporta mais que as pessoas parem de trabalhar tão cedo.

In [19]:
from torch import nn
from torch.nn import functional as F

class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(len(vocab), 128),
            nn.ReLU(),

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, len(categories)),
            #nn.ReLU(),

            #nn.Linear(64, 4),
        )

    def forward(self, X_batch):
        return self.seq(X_batch)

In [20]:
text_classifier = TextClassifier().cuda()
for X, Y in train_loader:
    Y_preds = text_classifier(X)
    print(Y_preds.shape)
    break

torch.Size([256, 9])


In [21]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score


def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).cpu().mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.cpu().detach().numpy(), Y_preds.cpu().detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

In [22]:
from torch.optim import Adam

epochs = 3
learning_rate = 1e-4

loss_fn = nn.CrossEntropyLoss()
text_classifier = TextClassifier().cuda()

optimizer = Adam(text_classifier.parameters(), lr=learning_rate)

TrainModel(text_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

100%|██████████| 211/211 [01:05<00:00,  3.24it/s]


Train Loss : 0.762
Valid Loss : 0.275
Valid Acc  : 0.940


100%|██████████| 211/211 [01:05<00:00,  3.22it/s]


Train Loss : 0.201
Valid Loss : 0.146
Valid Acc  : 0.970


100%|██████████| 211/211 [01:07<00:00,  3.14it/s]


Train Loss : 0.121
Valid Loss : 0.109
Valid Acc  : 0.978


In [23]:

import gc
def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds).cpu(), torch.cat(Y_shuffled).cpu()

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()


In [24]:

Y_actual, Y_preds = MakePredictions(text_classifier, test_loader)

In [25]:
pd.Series(Y_preds).unique()

array([7, 8, 6, 4, 1, 3, 5], dtype=int64)

In [26]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=categories))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

Test Accuracy : 0.9779807692307693

Classification Report : 
              precision    recall  f1-score   support

celebridades       0.00      0.00      0.00         9
     ciencia       0.96      0.95      0.96       265
      cinema       0.00      0.00      0.00         3
      comida       0.98      0.93      0.95        87
    educacao       0.97      0.94      0.95       394
    eleicoes       1.00      0.93      0.96        55
     esporte       0.99      0.99      0.99      1561
     mercado       0.98      0.98      0.98      4534
       poder       0.97      0.98      0.98      3492

    accuracy                           0.98     10400
   macro avg       0.76      0.74      0.75     10400
weighted avg       0.98      0.98      0.98     10400


Confusion Matrix : 
[[   0    0    0    0    0    0    0    6    3]
 [   0  253    0    0    3    0    0    8    1]
 [   0    0    0    0    0    0    0    2    1]
 [   0    1    0   81    0    0    0    4    1]
 [   0    2    0    0

d:\Projetos\Python\Artigos\venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Projetos\Python\Artigos\venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Projetos\Python\Artigos\venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [27]:
MakePredictions(text_classifier, DataLoader([[8, 'O ministro Alexandre de Moraes, do STF (Supremo Tribunal Federal), ignorou parecer da PGR (Procuradoria-Geral da República) e mandou prender, nesta quinta-feira (29), empresários do Distrito Federal suspeitos de financiar os atos golpistas de 8 de janeiro.']], batch_size=256, collate_fn=vectorize_batch))

(array([8], dtype=int64), array([8], dtype=int64))

In [28]:
torch.save(text_classifier, 'TextClassifierModel.pt')

In [29]:
import torch
model = torch.load('TextClassifierModel.pt')

In [30]:
from torch.utils.data import DataLoader

MakePredictions(model, DataLoader([[7, 'O presidente Luiz Inácio Lula da Silva (PT), o ministro Fernando Haddad (Fazenda) e o vice-presidente Geraldo Alckmin disseram, nesta sexta-feira (1º), que o governo foi surpreendido positivamente com o crescimento da economia em 2023 após o resultado do PIB (Produto Interno Bruto) superar as expectativas do início do mandato.']], batch_size=256, collate_fn=vectorize_batch))

(array([7], dtype=int64), array([7], dtype=int64))